<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [ ]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [6]:
# use requests.get() method with the provided static_url
# assign the response to a object
response = requests.get(static_url)
response

<Response [200]>

Create a `BeautifulSoup` object from the HTML `response`


In [7]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
soup = BeautifulSoup(response.content, 'html.parser')

Print the page title to verify if the `BeautifulSoup` object was created properly 


In [9]:
# Use soup.title attribute
title = soup.title
title

<title>List of Falcon 9 and Falcon Heavy launches - Wikipedia</title>

### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [11]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
html_tables = soup.find_all('table')
for i, table in enumerate(html_tables):
     print(f'Table {i + 1}:')
print(table)


Table 1:
Table 2:
Table 3:
Table 4:
Table 5:
Table 6:
Table 7:
Table 8:
Table 9:
Table 10:
Table 11:
Table 12:
Table 13:
Table 14:
Table 15:
Table 16:
Table 17:
Table 18:
Table 19:
Table 20:
Table 21:
Table 22:
Table 23:
Table 24:
Table 25:
Table 26:
<table class="nowraplinks navbox-subgroup" style="border-spacing:0"><tbody><tr><th class="navbox-group" scope="row" style="width:1%">General</th><td class="navbox-list-with-group navbox-list navbox-even" style="width:100%;padding:0"><div style="padding:0 0.25em">
<ul><li><a href="/wiki/List_of_crewed_spacecraft" title="List of crewed spacecraft">Crewed spacecraft</a>
<ul><li><a href="/wiki/List_of_human_spaceflights#Timeline" title="List of human spaceflights">timeline</a></li>
<li><a href="/wiki/Human_spaceflight_programs" title="Human spaceflight programs">by program</a></li></ul></li>
<li><a href="/wiki/List_of_human_spaceflights" title="List of human spaceflights">Spaceflights</a>
<ul><li><a href="/wiki/List_of_human_spaceflights,_1961

Starting from the third table is our target table contains the actual launch records.


In [12]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11"><span class="cite-bracket">[</span>b<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12"><span class="cite-bracket">[</span>c<span class="cite-bracket">]</span></a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [25]:
th_elements = first_launch_table.find_all('th')
print(th_elements)
column_names = []
for th in th_elements:
    name = extract_column_from_header(th)
    if name is not None and len(name) > 0:
            column_names.append(name)
            print("Column Names:", column_names)
    else:
     print(f'Error: {response.status_code}')


# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names


[<th scope="col">Flight No.
</th>, <th scope="col">Date andtime ()
</th>, <th scope="col"> 
</th>, <th scope="col">Launch site
</th>, <th scope="col">Payload
</th>, <th scope="col">Payload mass
</th>, <th scope="col">Orbit
</th>, <th scope="col">Customer
</th>, <th scope="col">Launchoutcome
</th>, <th scope="col">
</th>, <th rowspan="2" scope="row" style="text-align:center;">1
</th>, <th rowspan="2" scope="row" style="text-align:center;">2
</th>, <th rowspan="2" scope="row" style="text-align:center;">3
</th>, <th rowspan="3" scope="row" style="text-align:center;">4
</th>, <th rowspan="2" scope="row" style="text-align:center;">5
</th>, <th rowspan="2" scope="row" style="text-align:center;">6
</th>, <th rowspan="2" scope="row" style="text-align:center;">7
</th>]
Column Names: ['Flight No.']
Column Names: ['Flight No.', 'Date and time ( )']
Error: 200
Column Names: ['Flight No.', 'Date and time ( )', 'Launch site']
Column Names: ['Flight No.', 'Date and time ( )', 'Launch site', 'Payload'

Check the extracted column names


In [26]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [27]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [37]:
extracted_row = 0
launch_dict = []  # Ensure this is initialized

# Extract each table 
for table in soup.find_all('table', "wikitable plainrowheaders collapsible"):
    # Get table rows 
    for rows in table.find_all("tr"):
        # Check to see if the first table heading is a number corresponding to launch number 
        if rows.th:
            flight_number = rows.th.string.strip() if rows.th.string else None
            flag = flight_number.isdigit() if flight_number else False
        else:
            flag = False

        # Get table elements 
        row = rows.find_all('td')

        # If it is a number, save cells in a dictionary 
        if flag:
            extracted_row += 1
            launch_dict_entry = {}  # Create a new dictionary entry for each row

            # Flight Number value
            launch_dict_entry['Flight No.'] = flight_number
            
            # Date and Time values
            datatimelist = date_time(row[0]) if row[0] else (None, None)
            date = datatimelist[0].strip(',') if datatimelist[0] else None
            time = datatimelist[1] if datatimelist[1] else None
            
            # Booster version
            bv = booster_version(row[1]) if row[1] else None
            if not bv and row[1].a:
                bv = row[1].a.string
            
            launch_dict_entry['Version Booster'] = bv
            
            # Launch Site
            launch_site = row[2].a.string if row[2].a else None
            launch_dict_entry['Launch Site'] = launch_site
            
            # Payload
            payload = row[3].a.string if row[3].a else None
            launch_dict_entry['Payload'] = payload
            
            # Payload Mass
            payload_mass = get_mass(row[4]) if row[4] else None
            launch_dict_entry['Payload mass'] = payload_mass
            
            # Orbit
            orbit = row[5].a.string if row[5].a else None
            launch_dict_entry['Orbit'] = orbit
            
            # Customer
            customer = row[6].a.string if row[6].a else None
            launch_dict_entry['Customer'] = customer
            
            # Launch outcome
            launch_outcome = list(row[7].strings)[0] if row[7].strings else None
            launch_dict_entry['Launch outcome'] = launch_outcome
            
            # Booster landing
            booster_landing = landing_status(row[8]) if row[8] else None
            launch_dict_entry['Booster landing'] = booster_landing

            # Append the launch_dict_entry to launch_dict
            launch_dict.append(launch_dict_entry)

# At this point, launch_dict contains all the extracted data


After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [38]:
print(launch_dict)

[{'Flight No.': '1', 'Version Booster': 'F9 v1.07B0003.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon Spacecraft Qualification Unit', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'SpaceX', 'Launch outcome': 'Success\n', 'Booster landing': 'Failure'}, {'Flight No.': '2', 'Version Booster': 'F9 v1.07B0004.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': 0, 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Booster landing': 'Failure'}, {'Flight No.': '3', 'Version Booster': 'F9 v1.07B0005.18', 'Launch Site': 'CCAFS', 'Payload': 'Dragon', 'Payload mass': '525 kg', 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success', 'Booster landing': 'No attempt\n'}, {'Flight No.': '4', 'Version Booster': 'F9 v1.07B0006.18', 'Launch Site': 'CCAFS', 'Payload': 'SpaceX CRS-1', 'Payload mass': '4,700 kg', 'Orbit': 'LEO', 'Customer': 'NASA', 'Launch outcome': 'Success\n', 'Booster landing': 'No attempt'}, {'Flight No.': '5', 'Version Booster': 'F9 v1.07B0007.18'

In [42]:
launch_df = pd.DataFrame(launch_dict)
launch_df

,Flight No.,Version Booster,Launch Site,Payload,Payload mass,Orbit,Customer,Launch outcome,Booster landing
0,1,F9 v1.07B0003.18,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,Failure
1,2,F9 v1.07B0004.18,CCAFS,Dragon,0,LEO,NASA,Success,Failure
2,3,F9 v1.07B0005.18,CCAFS,Dragon,525 kg,LEO,NASA,Success,No attempt\n
3,4,F9 v1.07B0006.18,CCAFS,SpaceX CRS-1,"4,700 kg",LEO,NASA,Success\n,No attempt
4,5,F9 v1.07B0007.18,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,No attempt\n
...,...,...,...,...,...,...,...,...,...
116,117,F9 B5B1051.10657,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success
117,118,F9 B5B1058.8660,KSC,Starlink,"~14,000 kg",LEO,SpaceX,Success\n,Success
118,119,F9 B5B1063.2665,CCSFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,Success
119,120,F9 B5B1067.1668,KSC,SpaceX CRS-22,"3,328 kg",LEO,NASA,Success\n,Success


We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
